In [1]:
from utils import process_ns3_csv
input_filenames =  [
    '20241119_st-100_N-50_BH-5_SF-0_L-15_uni-1',
    '20241119_st-100_N-50_BH-10_SF-0_L-15_uni-1',
    '20241119_st-100_N-50_BH-15_SF-0_L-15_uni-1',
    '20241119_st-100_N-50_BH-20_SF-0_L-15_uni-1',
    '20241119_st-100_N-100_BH-10_SF-0_L-20_uni-1',
    '20241119_st-100_N-100_BH-20_SF-0_L-20_uni-1',
    '20241119_st-100_N-100_BH-30_SF-0_L-20_uni-1',
    '20241119_st-100_N-100_BH-40_SF-0_L-20_uni-1',
    # 其他的 filenames...
]

input_log_files = [
    '/log1119_BH0.1_uniform_1515seed_12345.txt',
    '/log1119_BH0.2_uniform_1515seed_12345.txt',
    '/log1119_BH0.3_uniform_1515seed_12345.txt',
    '/log1119_BH0.4_uniform_1515seed_12345.txt',
    '/log1119_BH0.1_uniform_2020seed_12345.txt',
    '/log1119_BH0.2_uniform_2020seed_12345.txt',
    '/log1119_BH0.3_uniform_2020seed_12345.txt',
    '/log1119_BH0.4_uniform_2020seed_12345.txt',
]



for i in range(len(input_filenames)):
    filename = input_filenames[i]
    log_file = './dataset/' + filename + input_log_files[i]
    input_file = './dataset/' + filename + '/MonitorSnifferRx.csv'
    output_file = './dataset/' + filename + '/ProcessedMonitorSnifferRx_Time10.csv'
    # process_ns3_csv(input_file, output_file, log_file)

In [21]:
from utils import load_data
import numpy as np
import models 

In [42]:
#训练集1
np.set_printoptions(threshold=np.inf)  # 这将打印所有内容，去掉折叠
data_file ='./dataset/' + '20241119_st-100_N-100_BH-40_SF-0_L-20_uni-1' + '/ProcessedMonitorSnifferRx_Time10.csv'
train_processed_data, train_adj, train_T, train_group_num = load_data(1,data_file)
# print(train_processed_data[0].edge_attr)
edge_feature_size = train_processed_data[0].edge_attr.shape[1]
nodes_feature_size = train_processed_data[0].x.shape[1]

In [43]:
#训练集2
# train_processed_data = []
# train_adj = []
# train_T = []
# for i in range(len(input_filenames)):
#     filename = input_filenames[i]
#     data_file = './dataset/' + filename + '/ProcessedMonitorSnifferRx_Time10.csv'
#     processed_data, adj, T, _ = load_data(1,data_file)
#     train_processed_data.append(processed_data[0])
#     train_adj.append(adj[0])
#     train_T.append(T[0])
# train_group_num = len(input_filenames)

In [44]:
# 设置numpy的打印选项，显示更多行列


# 打印节点特征（processed_data.x.shape）
# print(train_processed_data[0].x)
# print(adj[12])
# print(T[12])

In [45]:
# 打印边连接关系（processed_data.edge_index）
# print(processed_data[-1].edge_index.numpy().size)

In [46]:
# 打印节点标签（processed_data.y）
np.set_printoptions(threshold=np.inf)
# print(processed_data[0].y.numpy())
# print(processed_data[0].x[:, 0:23].numpy())
# print(processed_data[5].y.numpy())
# print(processed_data[5].edge_index)
# print(processed_data[5].edge_weights)
# print(processed_data[5].x.numpy())

In [47]:
import torch
train_model = models.HypergraphModel(nodes_feature_size, edge_feature_size)
print(train_model)
# 查看可用的 GPU 数量
num_gpus = torch.cuda.device_count()
print(f"Available GPUs: {num_gpus}")

# 打印每个 GPU 的名称
for i in range(num_gpus):
    print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_model = train_model.to(device)

HypergraphModel(
  (layer1): HypergraphConvLayer(
    (conv): HypergraphConv(3, 32)
  )
  (edge_conv): GraphConvolution()
  (class_classifier): Sequential(
    (0): Linear(in_features=32, out_features=16, bias=True)
    (1): ReLU()
    (2): Linear(in_features=16, out_features=2, bias=True)
  )
  (norm1): CustomBatchNorm()
  (norm2): CustomBatchNorm()
)
Available GPUs: 1
GPU 0: NVIDIA A100 80GB PCIe


In [56]:
import time
import torch.optim as optim
from utils import accuracy, auc, precision, recall, f1_score
import torch.nn.functional as F
criteria = torch.nn.CrossEntropyLoss()
learning_rate = 0.001
weight_decay = 5e-4
optimizer = optim.Adam(train_model.parameters(), lr=learning_rate, weight_decay=weight_decay)
acc_measure = accuracy
train_end_index = train_group_num
print(train_end_index)
val_index = group_num - 1
def train(epoch, save_path):
    t = time.time()
    train_model.train()
    optimizer.zero_grad()
    # print(processed_data[0].x[0:23].shape)
    # print(processed_data[0].edge_weights.shape)
    # print(processed_data[0].edge_index.shape)
    for i in range(train_end_index):
        # print(i)
        output = train_model(train_processed_data[i].x.to(device),
                             train_processed_data[i].edge_index.to(device), 
                             train_processed_data[i].edge_weights.to(device), 
                             train_processed_data[i].edge_attr.to(device), 
                             train_adj[i].to(device),
                             train_T[i].to(device))
        # print(output)
        labels = train_processed_data[i].y.to(device)
        loss_train = criteria(output, labels)
        # print(output[:, 1])
        acc_train = acc_measure(output, labels)
        
        loss_train.backward(retain_graph=True)
        optimizer.step()

        
        train_model.eval()
        output = train_model(train_processed_data[val_index].x.to(device),
                             train_processed_data[val_index].edge_index.to(device), 
                             train_processed_data[val_index].edge_weights.to(device),
                             train_processed_data[val_index].edge_attr.to(device),
                             train_adj[val_index].to(device),
                             train_T[val_index].to(device))
        # print(output)
        labels = train_processed_data[val_index].y.to(device)
        loss_val = criteria(output, labels)
        acc_val = acc_measure(output, labels)
        auc_val = auc(output, labels)
        per_val = precision(output, labels)
        recall_val = recall(output, labels)
        f1_val = f1_score(output, labels)
        # print(
        #     'loss_train: {:.4f}'.format(loss_train.item()),
        #     'acc_train: {:.4f}'.format(acc_train.item()),
        #     'loss_val: {:.4f}'.format(loss_val.item()),
        #     'acc_val: {:.4f}'.format(acc_val.item()),
        #     'auc_val: {:.4f}'.format(auc_val),
        #     'per_val: {:.4f}'.format(per_val),
        #     'recall_val: {:.4f}'.format(recall_val),
        #     'f1_val: {:.4f}'.format(f1_val),
        #     'time: {:.4f}s'.format(time.time() - t))
    # 每个 epoch 结束后保存模型
    torch.save(train_model.state_dict(), save_path)
    # torch.save({
    #     'epoch': epoch,
    #     'model_state_dict': train_model.state_dict(),
    #     'optimizer_state_dict': optimizer.state_dict(),
    #     }, save_path)
    # print(f'Model saved to {save_path}')

    return loss_val.item()


10


In [57]:
filename = '20241119_st-100_N-50_BH-5_SF-0_L-15_uni-1'
test_data_file = './dataset/' + filename + '/ProcessedMonitorSnifferRx_Time10.csv'
test_data, test_adj, test_T, test_group_num = load_data(1,test_data_file)
# print(processed_data[0].edge_attr)
end_index = len(test_data)
print(end_index)
top_k = 10  # 选择前10个最佳的指标值

filenames = [
    '20241119_st-100_N-50_BH-5_SF-0_L-15_uni-1',
    '20241119_st-100_N-50_BH-10_SF-0_L-15_uni-1',
    '20241119_st-100_N-50_BH-15_SF-0_L-15_uni-1',
    '20241119_st-100_N-50_BH-20_SF-0_L-15_uni-1',
    '20241119_st-100_N-100_BH-10_SF-0_L-20_uni-1',
    '20241119_st-100_N-100_BH-20_SF-0_L-20_uni-1',
    '20241119_st-100_N-100_BH-30_SF-0_L-20_uni-1',
    '20241119_st-100_N-100_BH-40_SF-0_L-20_uni-1',
    # 其他的 filenames...
]
num_metrics = len(filenames)

dataset_results = {
    'losses': [[[] for _ in range(end_index)] for _ in range(num_metrics)],  # 多维，记录每个数据集的多个loss值
    'accuracies': [[[] for _ in range(end_index)] for _ in range(num_metrics)],
    'aucs': [[[] for _ in range(end_index)] for _ in range(num_metrics)],
    'precisions': [[[] for _ in range(end_index)] for _ in range(num_metrics)],
    'recalls': [[[] for _ in range(end_index)] for _ in range(num_metrics)],
    'f1_scores': [[[] for _ in range(end_index)] for _ in range(num_metrics)],
}

final_results = {
        'losses': [[] for _ in range(num_metrics)],
        'accuracies': [[] for _ in range(num_metrics)],
        'aucs': [[] for _ in range(num_metrics)],
        'precisions': [[] for _ in range(num_metrics)],
        'recalls': [[] for _ in range(num_metrics)],
        'f1_scores': [[] for _ in range(num_metrics)],
    }

def get_top_k_avg(metric_list, top_k=top_k):
    """获取指标列表中的前top_k个值的均值"""
    return np.mean(sorted(metric_list, reverse=True)[:top_k])
        
def test(testfilename):
    test_data_file = '../dataset/' + testfilename + '/ProcessedMonitorSnifferRx_Time10.csv'
    test_data, test_adj, test_T, test_group_num = load_data(1,test_data_file)
    
    train_model.eval()
    
    for i in range(end_index):
        output = train_model(test_data[i].x.to(device),
                             test_data[i].edge_index.to(device).to(device), 
                             test_data[i].edge_weights.to(device),
                             test_data[i].edge_attr.to(device),
                             test_adj[i].to(device),
                             test_T[i].to(device))
        labels = test_data[test_group_num-1].y.to(device)
        # print(output)
        # print(labels)
        loss_test = criteria(output, labels)
        acc_test = acc_measure(output, labels)
        auc_test = auc(output, labels)
        per_test = precision(output, labels)
        recall_test = recall(output, labels)
        f1_test = f1_score(output, labels)

        # 将当前指标添加到相应的数据集列表中
        dataset_results['losses'][filenames.index(testfilename)][i].append(loss_test.item())
        dataset_results['accuracies'][filenames.index(testfilename)][i].append(acc_test.item())
        dataset_results['aucs'][filenames.index(testfilename)][i].append(auc_test)
        dataset_results['precisions'][filenames.index(testfilename)][i].append(per_test)
        dataset_results['recalls'][filenames.index(testfilename)][i].append(recall_test)
        dataset_results['f1_scores'][filenames.index(testfilename)][i].append(f1_test)
        
        # print(
        #     "Dataset " +
        #     str(filenames.index(testfilename)) +
        #     " Test set " +
        #     str(i) +
        #     " results:",
        #     "loss= {:.4f}".format(loss_test.item()),
        #     "accuracy= {:.4f}".format(acc_test.item()),
        #     'auc_test: {:.4f}'.format(auc_test),
        #     'per_test: {:.4f}'.format(per_test),
        #     'recall_test: {:.4f}'.format(recall_test),
        #     'f1_test: {:.4f}'.format(f1_test))
    return acc_test.item()

def get_top_k_AVEresult(testfilename):


    # 对每个数据集（共10个数据集）计算最好的10个测试结果的均值
    for i in range(end_index):
        final_results['losses'][filenames.index(testfilename)].append(get_top_k_avg(dataset_results['losses'][filenames.index(testfilename)][i]))
        final_results['accuracies'][filenames.index(testfilename)].append(get_top_k_avg(dataset_results['accuracies'][filenames.index(testfilename)][i]))
        final_results['aucs'][filenames.index(testfilename)].append(get_top_k_avg(dataset_results['aucs'][filenames.index(testfilename)][i]))
        final_results['precisions'][filenames.index(testfilename)].append(get_top_k_avg(dataset_results['precisions'][filenames.index(testfilename)][i]))
        final_results['recalls'][filenames.index(testfilename)].append(get_top_k_avg(dataset_results['recalls'][filenames.index(testfilename)][i]))
        final_results['f1_scores'][filenames.index(testfilename)].append(get_top_k_avg(dataset_results['f1_scores'][filenames.index(testfilename)][i]))

    # 输出每个数据集的最终结果（前10个测试结果的均值）
    print("\nFinal results (average of top 10 for each dataset):")
    for i in range(end_index):
        print(f"Dataset {i + 1}:")
        print(f"  Average loss: {final_results['losses'][filenames.index(testfilename)][i]:.4f}")
        print(f"  Average accuracy: {final_results['accuracies'][filenames.index(testfilename)][i]:.4f}")
        print(f"  Average AUC: {final_results['aucs'][filenames.index(testfilename)][i]:.4f}")
        print(f"  Average Precision: {final_results['precisions'][filenames.index(testfilename)][i]:.4f}")
        print(f"  Average Recall: {final_results['recalls'][filenames.index(testfilename)][i]:.4f}")
        print(f"  Average F1-score: {final_results['f1_scores'][filenames.index(testfilename)][i]:.4f}")
        print()
    output_file = './dataset/' + testfilename + '/result.txt'
    with open(output_file, 'a') as f:
        f.write("Test Results:\n")
        f.write("="*80 + "\n")
        f.write("\nFinal results" + testfilename + "(average of top 10 for each dataset):\n")
        for i in range(end_index):
            f.write(f"Dataset {i + 1}:\n")
            f.write(f"  Average loss: {final_results['losses'][filenames.index(testfilename)][i]:.4f}\n")
            f.write(f"  Average accuracy: {final_results['accuracies'][filenames.index(testfilename)][i]:.4f}\n")
            f.write(f"  Average AUC: {final_results['aucs'][filenames.index(testfilename)][i]:.4f}\n")
            f.write(f"  Average Precision: {final_results['precisions'][filenames.index(testfilename)][i]:.4f}\n")
            f.write(f"  Average Recall: {final_results['recalls'][filenames.index(testfilename)][i]:.4f}\n")
            f.write(f"  Average F1-score: {final_results['f1_scores'][filenames.index(testfilename)][i]:.4f}\n")
            f.write("-"*80 + "\n")

10


In [59]:
import os
from tqdm import tqdm
# torch.autograd.set_detect_anomaly(True)

num_epochs = 1000
early_stopping = 30
val_watch = []
t_total = time.time()
current_time = time.strftime("%Y-%m-%d_%H-%M-%S", time.localtime())


save_path = './models/' + current_time + '/' + current_time + '.pth'
os.makedirs(os.path.dirname(save_path), exist_ok=True)  # 创建目录，如果目录已存在不抛出错误

        
for epoch in tqdm(range(num_epochs), desc="Training Epochs", unit="epoch"):
    val = train(epoch, save_path)  # 训练并获得结果
    if val is None:  # 检查是否返回了None值
        print(f"Warning: train(epoch) returned None for epoch {epoch}. Skipping this iteration.")
        continue
    
    val_watch.append(val)  # 将结果添加到val_watch
    # if ((num_epochs - epoch) < 30):
    for filename in filenames:
        test(filename)  # 调用测试函数
        
for filename in filenames:
    get_top_k_AVEresult(filename)

    # if epoch > early_stopping and val_watch[-1] > np.mean(val_watch[-(early_stopping + 1):-1]):
    #     print("Early stopping...")
    #     break



print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))
print("Printing the weights : ")

Training Epochs: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [2:32:29<00:00,  9.15s/epoch]


Final results (average of top 10 for each dataset):
Dataset 1:
  Average loss: 0.6638
  Average accuracy: 0.8980
  Average AUC: 0.8670
  Average Precision: 0.2826
  Average Recall: 1.0000
  Average F1-score: 0.3819

Dataset 2:
  Average loss: 1.1560
  Average accuracy: 0.8980
  Average AUC: 0.7350
  Average Precision: 0.1702
  Average Recall: 0.8200
  Average F1-score: 0.2647

Dataset 3:
  Average loss: 2.0273
  Average accuracy: 0.9204
  Average AUC: 0.8105
  Average Precision: 1.0000
  Average Recall: 1.0000
  Average F1-score: 0.5022

Dataset 4:
  Average loss: 10.7811
  Average accuracy: 0.8980
  Average AUC: 0.6880
  Average Precision: 0.2184
  Average Recall: 1.0000
  Average F1-score: 0.2682

Dataset 5:
  Average loss: 5.5510
  Average accuracy: 0.9020
  Average AUC: 0.6868
  Average Precision: 0.3399
  Average Recall: 1.0000
  Average F1-score: 0.2502

Dataset 6:
  Average loss: 7.8881
  Average accuracy: 0.8980
  Average AUC: 0.7009
  Average Precision: 0.1325
  Average Recal

In [ ]:
# filename = '20241119_st-100_N-100_BH-40_SF-0_L-20_uni-1'
# data_file = './dataset/' + filename + '/ProcessedMonitorSnifferRx.csv'
# test_data, adj, T, group_num = load_data(1,data_file)
# # print(processed_data[0].edge_attr)
# edge_feature_size = test_data[0].edge_attr.shape[1]
# nodes_feature_size = test_data[0].x.shape[1]
# end_index = len(test_data)
# # print(end_index)

In [ ]:
# 加载模型
# import os
# import time
# import torch.optim as optim
# from utils import accuracy, auc, precision, recall, f1_score
# import torch.nn.functional as F
# import torch
# from utils import load_data
# import numpy as np
# import models 
# acc_measure = accuracy
# save_path = './models/2024-12-09_10-20-23/2024-12-09_10-20-23.pth'
# test_model = models.HypergraphModel(nodes_feature_size, edge_feature_size)
# # test_model.load_state_dict(torch.load(save_path, weights_only=True))
# checkpoint = torch.load(save_path, weights_only=True)
# test_model.load_state_dict(checkpoint['model_state_dict'])
# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
# epoch = checkpoint['epoch']
# test_model.eval()  # 切换到评估模式

In [ ]:
# for i in range(end_index):
#     print(i)
#     output = test_model(test_data[i].x, test_data[i].edge_index, 
#                              test_data[i].edge_weights, test_data[i].edge_attr, adj[i], T[i])
#     print(output)
#     labels = test_data[i].y
#     acc_val = acc_measure(output, labels)
#     auc_val = auc(output, labels)
#     per_val = precision(output, labels)
#     recall_val = recall(output, labels)
#     f1_val = f1_score(output, labels)
#     print(
#         'acc_val: {:.4f}'.format(acc_val.item()),
#         'auc_val: {:.4f}'.format(auc_val),
#         'per_val: {:.4f}'.format(per_val),
#         'recall_val: {:.4f}'.format(recall_val),
#         'f1_val: {:.4f}'.format(f1_val))